# Better way 6. 한 슬라이스에 start, end, stride 를 함께 쓰지 말자

* `somelist[start:end:stride]` 를 쓰면 start 에서 시작해서, end 에서 끝나면서, stride 만큼 간격을 설정할 수 있다.

In [2]:
a = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
odds = a[::2]
evens = a[1::2]
print(odds)
print(evens)

['red', 'yellow', 'blue']
['orange', 'green', 'purple']


* 문제는 stride 문법이 종종 예상치 못한 동작을 해서 버그를 만들어내기도 한다는 점이다.
* 리스트를 역순으로 만들 때 보통 stride -1 로 넘겨준다.
  * 아래의 경우 문자열

In [3]:
x = b'mongoose'
y = x[::-1]
print(y)

b'esoognom'


* UTF-8 바이트 문자열로 인코드된 유니코드 문자에는 원하는 대로 동작하지 않는다.

In [4]:
w = '한글'
x = w.encode('utf-8')
y = x[::-1]
z = y.decode('utf-8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte

In [8]:
# test
'한글'[::-1]

'글한'

* stride 를 start, end 와 함께 쓰면 혼란스러울 수 있다.
* 특히 stride 가 음수인 경우엔 더욱 그렇다.

In [9]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print(a[::2])
print(a[::-2])

['a', 'c', 'e', 'g']
['h', 'f', 'd', 'b']


In [10]:
print(a[2::2])
print(a[-2::-2])
print(a[-2:2:-2])
print(a[2:2:-2])

['c', 'e', 'g']
['g', 'e', 'c', 'a']
['g', 'e']
[]


* stride 를 start, end 인덱스와 함께 사용하지 말자
* stride 를 써야 한다면 양수 값을 사용하고 start 와 end 인덱스는 생략하는 게 좋다.
* 꼭 함께 써야 한다면, stride 를 적용한 결과를 변수에 할당하고, 이 변수를 슬라이스 한 결과를 다른 변수에 할당해서 사용하자.

In [12]:
b = a[::2]
c = b[1:-1]
print(b, c)

['a', 'c', 'e', 'g'] ['c', 'e']


* 슬라이싱 후 스트라이딩 하면 데이터의 얕은 복사본 (shallow copy) 이 추가로 생긴다.
* 시간과 메모리가 충분하지 않다면 `itertools` 의 `isslice` 메서드를 고려하자
  * Better Way 46

### 핵심 정리
* 한 슬라이스에 start, end, stride 를 지정하면 매우 혼란스러울 수 있다.
* 슬라이스에 start 와 end 인덱스 없이 양수 stride 값을 사용하자. 음수 stride 값은 가능하면 피하는 게 좋다.
* 한 슬라이스에 start, end, stride 를 함께 사용하는 상황은 피하자. 파라미터 세 개를 사용해야 한다면 할당 두 개 (하나는 슬라이스, 다른 하나는 스트라이드) 를 사용하거나 내장 모듈 itertools 의 isslice 를 사용하자.

# Better way 7. map 과 filter 대신 리스트 컴프리헨션을 사용하자

* list comprehension: 한 리스트에서 다른 리스트를 만들어내는 간결한 문법.
* 예) 각 숫자의 제곱을 계산할 때

In [13]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
squares = [x**2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


* 인수가 하나뿐인 함수를 적용하는 상황이 아니면, 간단한 연산에는 list comprehension 이 map 보다 명확하다.

In [16]:
# same result
squares = map(lambda x: x ** 2, a)
print(squares)
print(list(squares))

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


* 예) 2로 나누어 떨어지는 숫자의 제곱만 계산한다고 할 때

In [17]:
even_squares = [x**2 for x in a if x % 2 == 0]
print(even_squares)

[4, 16, 36, 64, 100]


* map 과 filter 를 쓰면 읽기  어렵다.

In [18]:
alt = map(lambda x: x**2, filter(lambda x: x%2 == 0, a))
assert even_squares == list(alt)

* dictionary 와 set 에도 list comprehension 에 해당되는 문법이 있다.

In [19]:
chile_ranks = {'ghost': 1, 'habanero': 2, 'cayenne': 3}
rank_dict = {rank: name for name, rank in chile_ranks.items()}
chile_len_set = {len(name) for name in rank_dict.values()}
print(rank_dict)
print(chile_len_set)

{1: 'ghost', 2: 'habanero', 3: 'cayenne'}
{8, 5, 7}


### 핵심 정리
* 리스트 컴프리헨션은 추가적인 lambda 표현식이 필요 없어서 내장 함수인 map 이나 filter 를 사용하는 것보다 명확하다.
* 리스트 컴프리헨션을 사용하면 입력 리스트에서 아이템을 간단히 건너뛸 수 있다. map 으로는 filter 를 사용하지 않고서는 이런 작업을 못한다.
* 딕셔너리와 세트도 컴프리헨션 표현식을 지원한다.

# Better way 8. 리스트 컴프리헨션에서 표현식을 두 개 넘게 쓰지 말자

* list comprehension 은 다중 루프도 지원한다.
* 예) 리스트를 담고 있는 리스트 (행렬) 을 flat 하게 만든다고 할 때

In [20]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
flat = [x for row in matrix for x in row] # left to right
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


* 위는 간단하고, 읽기 쉽고, 합당한 다중 루프
* 또 다른 합당한 사용법: 
  * 입력 리스트의 레이아웃을 두 레벨로 중복해서 구성하는 것
* 예) 2차원 행렬의 각 셀에 있는 값의 제곱을 구한다고 할 때

In [21]:
squares = [[x**2 for x in row] for row in matrix]
print(squares)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


* `[ ]` 가 들어있어 그리 좋아보이진 않지만 이해하기는 쉬움
* 예) 위의 flat 표현식을 다른 루프에 (또!) 넣는다면 이제 여러 줄로 구분해야 할 정도로 길어진다.

In [22]:
my_lists = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]]
]

flat = [x for sublist1 in my_lists
        for sublist2 in sublist1
        for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


* 똑같은 내용을 일반 루프문으로 구현하면 이해하기 쉽다(들여쓰기도 있고!)
  * 여기서는 extend 를 써서 마지막 loop 를 생략했음

In [23]:
flat = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        flat.extend(sublist2)
        
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


* list comprehension 은 다중 if 조건을 지원한다.
  * 같은 루프 레벨에 여러 조건이 있으면 암시적인 and 표현식
* 예) 4보다 큰 짝수 값만 가지고 올 때

In [24]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [x for x in a if x > 4 if x % 2 == 0]
c = [x for x in a if x > 4 and x % 2 == 0] # same results
print(b)
print(c)

[6, 8, 10]
[6, 8, 10]


* 조건은 각 루프 레벨에서 for 표현식 뒤에 설정할 수 있다.
* 예) 행렬에서 row 의 합이 10 이상이고, 3으로 나누어지는 셀을 구할 때

In [25]:
matrix = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
filtered = [[x for x in row if x % 3 == 0]
            for row in matrix if sum(row) >= 10]
print(filtered)

[[6], [9]]


* 간단하지만, 이해하기 어렵다.
* 이런 표현식이 적합해 보이는 상황을 마주치더라도, 이런 list comprehension 은 피하라고 강력히 말하고 싶다.
  * 다른 사람들이 이해하기 어렵다.
  * 몇 줄을 절약하는 장점이 그렇게 크지 않다.

* **경험칙**: 표현식이 두 개를 넘어가면 피하는게 좋다.
  * 조건 두 개, 루프 두 개, 조건 한개와 루프 한 개 정도?
* 이것보다 복잡해지면, 일반적인 if / for 문을 사용하고 헬퍼 함수 (Better way 16) 을 작성하자.

### 핵심 정리
* 리스트 컴프리헨션은 다중 루프와 루프 레벨별 다중 조건을 지원한다.
* 표현식이 두 개가 넘게 들어있는 리스트 컴프리헨션은 이해하기 매우 어려우므로 피해야 한다.

# Better way 9. 컴프리헨션이 클 때는 제너레이터 표현식을 고려하자

* **list comprehension 의 문제점**: 
  * 입력 시퀀스에 있는 각 값 별로 아이템을 하나씩 담은 새 리스트를 통째로 생성한다.
  * 입력이 크면 메모리를 많이 소모해서 프로그램을 망가뜨리는 원인이 되기도 함
* 예) 파일을 읽고 각 줄에 있는 문자의 개수를 반환할 때
  * 리스트 컴프리헨션을 사용하는 경우 각 줄의 길이만큼 메모리가 필요
  * 파일에 오류가 있거나 끊김이 없는 네트워크 소켓일 경우 문제가 발생

In [26]:
# 입력값이 적은 경우만 처리할 수 있는 방식
value = [len(x) for x in open('my_file.txt')]
print(value)

[222, 93, 106, 258, 292, 161, 112, 29]


* **제너레이터 표현식 generator expression**
  * 실행될 때 출력 시퀀스를 모두 구체화 (여기서는 메모리에 로딩) 하지 않는다.
  * 대신 표현식에서 한 번에 한 아이템을 내 주는 (읽어 오는) 이터레이터 iterator 로 평가된다.

* 사용법:
  * `( )` 사이에 list comprehension 과 비슷한 문법을 사용
* 예) 이전 코드와 동일한 기능을 제너레이터 표현식으로 작성할 때

In [27]:
it = (len(x) for x in open('my_file.txt'))
print(it) # iterator!

<generator object <genexpr> at 0x1050b9728>


* 필요할 때 제너레이터 표현식에서 다음 출력을 생성하려면 내장 함수 next 로 반환받은 이터레이터를 *한 번 전진* 시키면 된다.
  * 메모리 사용량 걱정을 덜 수 있음

In [28]:
print(next(it))
print(next(it))

222
93


* 또 다른 강력한 결과:
  * 다른 제너레이터 표현식과 함께 사용할 수 있음
* 예) 앞의 제너레이터 변환식이 반환한 이터레이터를 다른 제너레이터 표현식의 입력으로 사용할 때

In [29]:
roots = ((x, x**0.5) for x in it)
print(next(roots))

(106, 10.295630140987)


* 제너레이터를 서로 연결하면 파이썬에서 매우 빠르게 실행할 수 있다.
* 큰 입력 스트림에 동작하는 기능을 결합하는 방법을 찾을 때는 제너레이터 표현식이 최선의 도구!
* 단, 이터레이터는 한 번 넘게 사용할 수 없음 (Better way 17)

### 핵심 정리
* 리스트 컴프리헨션은 큰 입력을 처리할 때 너무 많은 메모리를 소모해서 문제를 일으킬 수  있다.
* 제너레이터 표현식은 이터레이터로 한 번에 한 출력만 만드므로 메모리 문제를 피할 수 있다.
* 한 제너레이터 표현식에서 나온 이터레이터를 또 다른 제너레이터 표현식의 for 서브 표현식으로 넘기는 방식으로 제너레이터 표현식을 조합할 수 있다.
* 제너레이터 표현식은 서로 연결되어 있을 때 매우 빠르게 실행된다.

# Better way 10. range 보다는 enumerate 를 사용하자

* 내장 함수 range: 정수 집합을 순회 (iterate) 하는 루프를 실행할 때 유용함

In [32]:
from random import randint

random_bits = 0
for i in range(64):
    if randint(0, 1):
        random_bits |= 1 << i
print(random_bits)
print('%s' % bin(random_bits))
print('%s' % hex(random_bits))

14052340486991707367
0b1100001100000011111100010011100001101101111110101111110011100111
0xc303f1386dfafce7


* 문자열의 리스트를 순회할 때? 직접 루프 실행

In [33]:
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for flavor in flavor_list:
    print('%s is delicious' % flavor)

vanilla is delicious
chocolate is delicious
pecan is delicious
strawberry is delicious


* 예) 리스트를 순회하면서 리스트의 현재 아이템의 인덱스를 알고 싶은 경우
  * 한 가지 방법: range 사용 (비추천)

In [34]:
for i in range(len(flavor_list)):
    flavor = flavor_list[i]
    print('%d: %s' % (i+1, flavor))

1: vanilla
2: chocolate
3: pecan
4: strawberry


* 나쁜 점:
  * 세련되지 못해 보인다(?)
  * 리스트의 길이를 알아내야 하고, 배열을 인덱스로 접근해야 하며, 읽기 불편하다.

* 대안: enumerate 를 쓰자.
  * enumerate 는 지연 제너레이터 lazy generator 로 이터레이터를 감싼다.
  * 인덱스와 다음 값을 한 쌍으로 넘겨준다.

In [35]:
for i, flavor in enumerate(flavor_list):
    print('%d: %s' % (i, flavor))

0: vanilla
1: chocolate
2: pecan
3: strawberry


### 핵심 정리
* enumerate 는 이터레이터를 순회하면서 이터레이터에서 각 아이템의 인덱스를 얻어오는 간결한 문법을 제공한다.
* range 로 루프를 실행하고 시퀀스에 인덱스로 접근하기보다는 enumerate 를 사용하는 게 좋다.
* enumerate 에 두 번째 파라미터를 사용하면 세기 시작할 숫자를 지정할 수 있다 (기본 값은 0이다).